In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler                          
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationSummaryBufferMemory
from langchain.schema.runnable import RunnablePassthrough

# 이 과제는 대화를 배제하고 FewShotPromptTemplate 을 활용하여 형식을 만들어 준다
# FewShotPromptTemplate으로 할려다가 못했다. 방식이 다르다. 각잡고 할려다가 시간이 오래 걸릴꺼 같아서(시행착오) 다음 기회로 미룬다 
# MessagesPlaceholder를 사용하는 방식이 FewShotChatMessagePromptTemplate 과는 다르게 구성 해야 한다 코드 리뷰에서 물어 보도록 한다 
# 역시 개념을 파악 하기가 녹록 하지 않다

llm = ChatOpenAI(                                          
    #model="gpt-4o-mini-2024-07-18",                     # 이모지 동작이 안된다 워닝 뜨면서 안된다
    model="gpt-3.5-turbo",                               # 혹시나 해서 모델을 지정해 준다 (gpt-4o-mini-2024-07-18 노마드코더 노마드코더 파이썬 챌린지 코드리뷰 보다가 이 사실을 알게 되었다... 세상은 정보의 네트워크로 되어 있다)
    temperature=1,                                       # 성능 쯤으로 생각하자 (노마드코더 파이썬 챌린지 코드리뷰에서 창의성이라는것을 알았다.)    
    # streaming=True,                                        # 체인의 호출에 따른 내용을 스트리밍 즉 과정을 보여준다
    # callbacks=[StreamingStdOutCallbackHandler()]           # 한글짜씩 보여달라고 요청한다
    # 답변이 간단해서 필요가 없다
    )

memory = ConversationSummaryBufferMemory(                  # 마지막 예제를 그대로 따른다
    llm=llm,
    max_token_limit=120,                                   # 120 이 정확하게 뭔지는 모르겠지만 비용이라는 것은 알겠다
    return_messages=True,                                  # 이걸 해 줘야 llm 이 알아 먹을 수 있다 안하면 걍 str 형식으로 저장하게 된다. 저장된 내용을 다시 전달 해 줘야 되지 않겠는가
)

examples = [                                               # 샘플의 형식이다
    {
        "movie": "아이언맨",
        "answer": "🎶💖✨"
    },
    {
        "movie": "기생충",
        "answer": "🤗👮‍♀️🤹‍♀️"
    },
    {
        "movie": "록키 호러 픽쳐 쇼",
        "answer": "🚞🛺🦽"
    }
]


example_prompt = ChatPromptTemplate.from_messages(             # 11 단원과 같다
    [
        ("human", "{movie}를 이모지콘 3개로 표현 해 줘"),
        ("ai", "{answer}")
    ]
)

example_prompt_form = FewShotChatMessagePromptTemplate(             # 전과 같다 대화형식의 템플릿이라고 보면 된다 일반 템플릿은 구동의 구조가 다르다
    examples=examples,                                              # 질문과 답변의 샘플이다. 이걸 템플릿에 입력해준다 
    example_prompt=example_prompt                                   # 윗줄의 샘플을 가지고 만드는 프롬프트 템플릿의 구조 이다 단순하게 물어보면 답변하는 형식이다
    
)

final_prompt = ChatPromptTemplate.from_messages(                                                                     # 이 함수로 다양하게 최종 템플릿에 추가가 가능하다. arg 가 다양하다. 그래서 여러가지를 가지고 참고가 가능하다
    [
        ("system", "당신은 영화를 3개의 이모지로 표현하는 영화 이모지 전문가"),                                                 # 자아를 입히는 과정이다
        MessagesPlaceholder(variable_name="history"),                                                                # 형식 참조후 메세지 플레이스 홀더에서 history 라는 키값의 value 를 불러 온다
        example_prompt_form,                                                                                         # 이넘아는 오로지 형식을 참고하는 템플릿 이다.... form 이 가지고 있는 샘플과 샘플 프롬프트를 참고 한다   
        ("human", "{movie}")                                                                                         # 입력의 형태를 만들어 준다 
    ]  
)


def load_memory(_):                                                                                                  # history 라는 키의 메모리를 불러와서 리턴하는 함수 형태가 괴상하다                                                                               
    return memory.load_memory_variables({})["history"]                                                               

chain = RunnablePassthrough.assign(history = load_memory ) | final_prompt | llm                                      # 너무 생소하다 RunnablePassthrough 데이터를 전달하는 클라스에서 history 라는 변수는 load_memory라는 함수고 () 없이 쓴다 게다가 원형에 매개변수가 꼭 필요하다고 한다 너무 생소함 그냥 외워두자
                                                                                                                     # 아미도 실행이 가능해야 하기때문인건지 모르겠다 때문에 위의 함수와 순서가 바뀌면 안된다

def invoke_chain(movie):                                                                                             # 메모리에 결과값을 전달하기 위하여 함수를 사용했다 매개변수는 아마 str 이겠다
    result = chain.invoke({"movie": movie})                                                                          # 아우풋을 만들어 내서 result 에 담고
    memory.save_context(                                                                                             # 메모리에 저장한다
        {"input": movie},                                                                                            # 딕셔너리 형태로 2개의 콘텍스트를 너을 수 있다 인풋에는 매개변수 입력값을
        {"output": result.content}                                                                                   # 아웃풋에는 결과값을 넣는데 맞는 형식을 너어줘야 한다 걍 너으면 에러남           
    )
    print(result.content)   ## 보여준다


invoke_chain("아이언맨")


💥🦸‍♂️🤖


In [3]:
invoke_chain("수퍼맨")                 # 잘 나오는데 가끔 /***** 과 같은 코드가 나올때가 있다

🦸‍♂️💫🌌


In [2]:
invoke_chain("스파이더맨")

🕷️🕸️🦸‍♂️


In [4]:
invoke_chain("아이언맨")                 # 역시 같은 이모지가 나온다 동작이 잘 된다

💥🦸‍♂️🤖
